This note replicates the results in Imrohoroğlu, A. (1989). Cost of business cycles with indivisibilities and liquidity constraints. Journal of Political economy, 97(6), 1364-1383.

In [ ]:
from numba import njit, prange, float64, int64
from numba.experimental import jitclass
from interpolation import interp
from quantecon.optimize import brent_max, brentq
from quantecon import MarkovChain

## Calibrate Parameters and Discretize State Variables

In [ ]:
class BusinessCycleModel:
    def __init__(self,
                 period = 6, # weeks
                 r_save = .00, # net real return on stored assets
                 r_borrow = .08, # rate on borrowing annually
                 y = 1, # income if employed
                 theta = .25, # income ratio if unemployed
                 beta = .995, # implies an annual time discount rate of 4%
                 sigma = 1.5, # coefficient of risk aversion
                 business_cycle=True, 
                 a_max = 8, 
                 a_min = 0,
                 Na = 301,
                 verbose = True
                ):
        
        # parameters
        self.period, self.beta = period, beta
        self.periods_in_a_year = 52 / period
        self.R_save = 1 + r_save
        self.R_borrow = (1 + r_borrow)**(1/self.periods_in_a_year)
        self.sigma = sigma
        self.Na = Na
        self.a_min, self.a_max = a_min, a_max
        
        # transition matrices
        if business_cycle:
            self.P = np.array([
                        [0.9141, 0.0234, 0.0587, 0.0038],
                        [0.5625, 0.3750, 0.0269, 0.0356],
                        [0.0608, 0.0016, 0.8813, 0.0563],
                        [0.0375, 0.0250, 0.4031, 0.5344],
                    ])
            self.Ns = 4 
            self.Ns_label = ['ge', 'gu', 'be', 'bu']
            if verbose:
                print(f'Construct model with business cycle.')
        else:
            self.P = np.array([
                        [.9565, .0435],
                        [.5000, .5000]
                    ])
            self.Ns = 2 
            self.Ns_label = ['e', 'u']
            if verbose:
                print(f'Construct model without business cycle.')
        
        # state variables
        self.a_vals = np.linspace(a_min,a_max,self.Na)
        self.y_vals = np.array([y, y*theta]*int(self.Ns/2))    

## Value Function Iteration

In [ ]:
def operator_factory_value_iteration(bcm, parallel_flag=True):
    """
    A function factory that output utility function and value fuction iterator
    """
    
    beta, sigma  = bcm.beta, bcm.sigma
    R_save, R_borrow = bcm.R_save, bcm.R_borrow
    Na, Ns = bcm.Na, bcm.Ns
    a_vals, y_vals = bcm.a_vals, bcm.y_vals
    P = bcm.P
    _u = np.empty([Na, Ns, Na])
    
    @njit
    def R_func(a):
        if a >= 0:
            R = R_save
        else:
            R = R_borrow
        return R  
    
    @njit
    def U_func(c, sigma):
        if sigma == 1:
            u = np.log(c)
        else:
            u = c**(1-sigma) / (1-sigma)
        return u

    @njit(parallel=parallel_flag)
    def util():
        """
        Indirect utility function
        Calculate at first for tabulation use later
        """
        c = np.empty_like(_u)
        u = np.empty_like(_u)
        for a0 in prange(Na):
            for s0 in prange(Ns):
                for a1 in prange(Na):
                    R = R_func(a_vals[a1])
                    c0 = a_vals[a0] + y_vals[s0] - a_vals[a1]/R
                    c[a0, s0, a1] = c0
                    if c0 <= 0:
                        u[a0, s0, a1] = -1e20
                    else:
                        u[a0, s0, a1] = U_func(c0, sigma)
        return c, u

    @njit(parallel=parallel_flag)
    def T(v, u, c):
        """
        The Bellman operator
        Return new value function and the policy function
        """

        v_new = np.empty_like(v)
        policy = np.empty_like(v)
        gamma = np.empty_like(v)
        for a0 in prange(Na):
            for s0 in prange(Ns):
                Na1 = np.argwhere(c[a0,s0,:]>0).max() # the largest a1 that c0>0
                v_vals = np.empty(Na1+1)
                for a1 in prange(Na1+1):
                    u0 = u[a0,s0,a1]
                    v_vals[a1] = u0 + beta * P[s0] @ v[a1,:]
                v_new[a0,s0] = np.max(v_vals)
                a1 = np.argmax(v_vals)
                policy[a0,s0] = a1
                
                R = R_func(a_vals[a1])
                gamma[a0,s0] = a_vals[a0] + y_vals[s0] - a_vals[a1]/R
                
        return v_new, policy, gamma

    return util, T

In [ ]:
def solve_model_value_iteration(
                bcm, 
                use_parallel = True,
                tol=1e-5, 
                max_iter=5000,
                verbose=True,
                print_skip=100):
    """
    Iterates to convergence on the Bellman equations
    """
    if verbose:
        print('Solve model with value function iteration.')
    
    util, T = operator_factory_value_iteration(bcm, use_parallel)
    c, u = util() 
    
    # Set up loop
    i = 0
    error = tol + 1
    
    # Initialize v
    v = np.zeros([bcm.Na, bcm.Ns])    
        
    while i < max_iter and error > tol:
        v_new, policy, gamma = T(v,u,c)
        error = np.max(np.abs(v - v_new))
        i += 1
        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")
        v = v_new
        
    if i == max_iter:
        print("Failed to converge!")

    if i < max_iter and verbose:
        print(f"Converged in {i} iterations.") 
                
    return v, policy.astype(int), gamma 

- solve the default economy with stroage technology, i.e. $a>0$

In [ ]:
bc_storage = BusinessCycleModel()

In [ ]:
%%time
v, policy, gamma = solve_model_value_iteration(bc_storage,)

In [ ]:
# plot policy function
def plot_asset_policy_function(bcm, policy):
    fig, ax = plt.subplots()
    for s in range(bcm.Ns):
        if policy.dtype == "int64":
            a1_vals = bcm.a_vals[policy[:,s]]
        else:
            a1_vals = policy[:,s]
        ax.plot(bcm.a_vals,a1_vals, label=bcm.Ns_label[s])
    ax.plot(bcm.a_vals,bcm.a_vals, color='black', linestyle='--')
    ax.set(xlabel='$a_t$', ylabel='$a_{t+1}$')
    ax.legend()

plot_asset_policy_function(bc_storage, policy)

In [ ]:
# plot consuming policy function
def plot_consuming_policy_function(bcm, gamma):
    fig, ax = plt.subplots()
    for s in range(bcm.Ns):
        label = rf'$\sigma^*(\cdot, {s}) - {bcm.Ns_label[s]}$'
        ax.plot(bcm.a_vals, gamma[:, s], label=label)
    ax.set(xlabel='$a_t$', ylabel='$c_t$')
    ax.legend()

plot_consuming_policy_function(bc_storage, σ_star)

- we now test the economy with intermediation technology, i.e. $a > -B$

In [ ]:
%%time
bc_borrow = BusinessCycleModel(a_min = -8, Na = 601)
v, policy, gamma  = solve_model_value_iteration(bc_borrow)

In [ ]:
plot_asset_policy_function(bc_borrow, policy)

In [ ]:
plot_consuming_policy_function(bc_borrow, gamma)

## Policy Function Iteration

- policy function iteration is faster and more accurate than value function iteration, probably due to the utilitzation of firt order condition
- however policy function iteration seems to be unavailable in the economy with storage technology and a different borrowing rate

In [ ]:
def operator_factory_time_iteration(bcm):
    """
    A function factory that output utility function and value fuction iterator
    """
    
    beta, sigma = bcm.beta, bcm.sigma
    R = bcm.R_save
    Na, Ns = bcm.Na, bcm.Ns
    a_vals, y_vals = bcm.a_vals, bcm.y_vals
    P = bcm.P      
    
    @njit
    def u_prime(c):
        return c**(-sigma)

    @njit
    def euler_diff(c, a, s, gamma_vals):
        """
        The difference between the left- and right-hand side
        of the Euler Equation, given current policy gamma.
        """
        
        # Convert policy into a function by linear interpolation
        def gamma(a, s):
            return interp(a_vals, gamma_vals[:, s], a)

        # Calculate the expectation conditional on current z
        expect = 0.0
        for s1 in range(Ns):
            expect += u_prime(gamma(R * (a - c + y_vals[s]), s1)) * P[s, s1]
        
        diff = u_prime(c) - max(beta * R * expect, u_prime(a+y_vals[s]))

        return diff
    
    @njit
    def K(gamma):
        """
        The operator K.
        """
        gamma_new = np.empty_like(gamma)
        asset_policy = np.empty_like(gamma)
        for i, a in enumerate(a_vals):
            for s in range(Ns):
                result = brentq(euler_diff, 1e-8, a+y_vals[s], args=(a, s, gamma))
                c = result.root
                gamma_new[i, s] = c
                asset_policy[i, s] = R * (a - c + y_vals[s]) 

        return gamma_new, asset_policy
    
    return euler_diff, K

In [ ]:
def solve_model_time_iter(bcm,    # Class with model information
                          tol=1e-4,
                          max_iter=1000,
                          verbose=True,
                          print_skip=25):
    
    if verbose:
        print('Solve model with policy function iteration.')
    
    euler_diff, K = operator_factory_time_iteration(bcm)
        
    # Set up initial consumption policy
    gamma = np.repeat(bcm.a_vals.reshape(bcm.Na, 1), bcm.Ns, axis=1)

    # Set up loop
    i = 0
    error = tol + 1

    while i < max_iter and error > tol:
        gamma_new, asset_policy = K(gamma)
        error = np.max(np.abs(gamma - gamma_new))
        i += 1
        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")
        gamma = gamma_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose:
        print(f"\nConverged in {i} iterations.")

    return gamma_new, asset_policy

In [ ]:
%%time
bc_storage = BusinessCycleModel()
gamma, asset_policy = solve_model_time_iter(bc_storage)

In [ ]:
plot_consuming_policy_function(bc_storage, gamma)

In [ ]:
plot_asset_policy_function(bc_storage, asset_policy)

## Stationary Distribution - Montel Carlo Simulation

In [ ]:
def compute_asset_series(bcm, gamma_star, T=500_000, seed=1234):
    """
    Simulates a time series of length T for assets, given optimal
    savings behavior.
    """
    P, a_vals, y_vals, R  = bcm.P, bcm.a_vals, bcm.y_vals, bcm.R_save
     
    gamma = lambda a, s: interp(a_vals, gamma_star[:, s], a)

    # Simulate the exogeneous state process
    mc = MarkovChain(P)
    s_seq = mc.simulate(T, random_state=seed)

    # Simulate the asset path
    a_path = np.zeros(T+1)
    c_path = np.zeros(T)
    for t in range(T):
        s = s_seq[t]
        c_path[t] = gamma(a_path[t], s)
        a_path[t+1] = R * (a_path[t] - gamma(a_path[t], s) + y_vals[s])
    return a_path, c_path, s_seq

In [ ]:
bc_storage = BusinessCycleModel()
gamma, _ = solve_model_time_iter(bc_storage, verbose=False)
a_path, c_path, s_seq = compute_asset_series(bc_storage, gamma)

In [ ]:
fig, ax = plt.subplots()
for i in range(nbc.Ns):
    ax.hist(a_path[np.argwhere(s_seq == i)], 
                 label=nbc.Ns_label[i],
                 bins=100, alpha=0.5, density=True,)
ax.legend()
plt.show();

## Stationary Distribution - Iteration

In [ ]:
def solve_invariant_dist(
                bcm, 
                policy,
                tol=1e-7, 
                max_iter=2000,
                verbose=True,
                print_skip=50):
    
    Na, Ns = bcm.Na, bcm.Ns
    P = bcm.P
    
    # Set up loop
    i = 0
    error = 1+tol
    
    # Initialize distribution
    pmf = np.ones_like(policy) * (1/(Na*Ns))
    
    # Distirbution iteration
    @njit
    def dist_iter(pmf):
        pmf_new = np.zeros_like(pmf)
        for a0 in prange(Na):
            for s0 in prange(Ns):
                a1 = policy[a0,s0]
                for s1 in prange(Ns):
                    pmf_new[a1,s1] += P[s0,s1] * pmf[a0,s0]
        return pmf_new
        
    while i < max_iter and error > tol:
        pmf_new = dist_iter(pmf)
        error = np.max(np.abs(pmf - pmf_new))
        i += 1
        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")
#             print(f"Pmf sum is {pmf_new.flatten().sum_()}.")
        pmf = pmf_new
        
    if i == max_iter:
        print("Failed to converge!")

    if i < max_iter and verbose:
        print(f"Converged in {i} iterations.")        
    
    return pmf

In [ ]:
bc_storage = BusinessCycleModel()
_, policy, _ = solve_model_value_iteration(bc_storage, verbose=False)
pmf = solve_invariant_dist(bc_storage, policy)

In [ ]:
# plot distribution
def plot_invariant_distribution(pmf, bcm):
    fig, ax = plt.subplots()
    for i in range(bcm.Ns):
        ax.plot(bcm.a_vals, pmf[:,i], label=bcm.Ns_label[i])
        ax.legend();

plot_invariant_distribution(pmf, bc_storage)

## Economy with Perfect Insurance

In [ ]:
class BCM_PI(BusinessCycleModel):
    def generate_mc_series(self, 
                           T= 500_000, 
                           num_reps = 1_000, 
                           seed= 1234, ):
        
        mc = MarkovChain(self.P, state_values=self.y_vals)
        s_seqs = mc.simulate(T, random_state=seed, num_reps=num_reps)
        return s_seqs
    
    def calculate_discounted_utilities(self,):
        ss_average_income = self.generate_mc_series().mean(axis=0).mean()
        ss_average_utility = ss_average_income ** (1-self.sigma) / (1-self.sigma)
        return ss_average_utility/(1-self.beta)        
        

In [ ]:
def calculate_consumption_compensation(sigma = 1.5):

    bc_pi = BCM_PI(business_cycle=True, sigma=sigma)
    v_bcpi = bc_pi.calculate_discounted_utilities()

    nbc_pi = BCM_PI(business_cycle=False, sigma=sigma)
    v_nbcpi = nbc_pi.calculate_discounted_utilities()

    mu = (v_nbcpi/v_bcpi)**(1/(1-sigma)) -1
    return mu

calculate_consumption_compensation()

## Results on Consumption Lose 

In [ ]:
def compute_consumption_loss(sigma, a_max, Na):
    bc = BusinessCycleModel(sigma=sigma, a_max=a_max, Na=Na,
                            verbose=False)
    v, policy, _ = solve_model_value_iteration(bc, verbose=False)
    pmf = solve_invariant_dist(bc, policy, verbose=False)    
    v_bc = np.sum(pmf * v)
    
    nbc = BusinessCycleModel(sigma=sigma, a_max=a_max, Na=Na,
                             business_cycle=False, verbose=False)
    v, policy, _ = solve_model_value_iteration(nbc, verbose=False)
    pmf = solve_invariant_dist(nbc, policy, verbose=False)    
    v_nbc = np.sum(pmf * v)  
    
    if sigma == 1:
        mu = np.e**((v_nbc-v_bc) * (1-bc.beta)) - 1
    else:
        mu = (v_nbc/v_bc)**(1/(1-sigma)) -1
    return mu

In [ ]:
for sigma, a_max, Na in zip([1.5, 6.2], [8, 16],[301, 601]):
    mu = compute_consumption_loss(sigma, a_max=a_max, Na=Na,)
    print(f'When sigma = {sigma} , the consumption loss : {mu:.2%}')